## Database Setup
### Outline
- Create a database and establish a connection
- Create tables
- Load datasets
- Export datasets to MySQL tables

### Additional Considerations
- column names from governance_indicators.csv can be used directly
- OECD column names can be used
- economic freedom scores as well
- SEZ is okay

In [1]:
%pip install mysql-connector-python
%pip install mysql

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\Harb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: C:\Users\Harb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import os

env = os.environ

MYSQL_CONFIG = {
    'host': env.get("MYSQL_HOST"),
    'user': env.get("MYSQL_USER"), 
    'password': env.get("MYSQL_PW"),
    'database': env.get("MYSQL_DB")
}

connection = mysql.connector.connect(
            host=MYSQL_CONFIG['host'],
            user=MYSQL_CONFIG['user'],
            password=MYSQL_CONFIG['password']
        )

governance_indicators = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\governance_indicators.csv")
economic_freedom_scores = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\economic_freedom_country_scores.csv")
oecd_fdi_data = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\OECD_fdi_flows_fdi_restrictiveness.csv")
global_sez = pd.read_csv("C:\\Users\\Harb\\OneDrive\\Documents\\foreign-direct-investment-analysis\\data\\processed\\political_factors\\global_special_economic_zones.csv")



def create_table_from_dataframe(cursor, table_name, dataframe):

    columns = ', '.join([f"`{col}` TEXT" if dtype == 'object' else f"`{col}` DOUBLE" 
                         for col, dtype in dataframe.dtypes.items()])
    create_table_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({columns});"
    cursor.execute(create_table_sql)

    placeholders = ', '.join(['%s'] * len(dataframe.columns))
    insert_sql = f"INSERT INTO `{table_name}` VALUES ({placeholders})"

    for row in dataframe.itertuples(index=False, name=None):
        cursor.execute(insert_sql, row)

    print(f"Table '{table_name}' created")


def create_database():
    try:
        
        cursor = connection.cursor()
        
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {MYSQL_CONFIG['database']}")
        cursor.execute(f"USE {MYSQL_CONFIG['database']}")

        datasets = {
            "governance_indicators": governance_indicators,
            "economic_freedom_country_scores": economic_freedom_scores,
            "oecd_fdi_flows_restrictiveness": oecd_fdi_data,
        }

        for table_name, df in datasets.items():
            create_table_from_dataframe(cursor, table_name, df)

        connection.commit()

        print("Tables done.")


    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [ ]:
create_database()